<a href="https://colab.research.google.com/github/rhodes-byu/stat-486/blob/main/notebooks/02-gradient-descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><p><b></b></p>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

In [ ]:
url = 'https://github.com/rhodes-byu/stat-486/raw/refs/heads/main/data/icecreamcone2.csv'
df = pd.read_csv(url)
df.shape

In [ ]:
df.head(2)

In [ ]:
#features = df[['tempC','saltPct','waterPct']]

## For visualization purposes, I will only use one feature in this code
features = df[['waterPct']]
features = StandardScaler().fit_transform(features)
target = df['logBV']
n = len(target)

In [ ]:
### Use Linear Regression from sklearn
rounding_level = 4

lm = LinearRegression()
lm.fit(features, target)
beta_LR = [lm.intercept_.round(rounding_level)] + list(lm.coef_.round(rounding_level))
print(beta_LR)

In [ ]:
# Add the intercept term to the X matrix for normal-equations computation
X = PolynomialFeatures(degree=1).fit_transform(features)
y = target.values.reshape(n,1)
X.shape, y.shape

In [ ]:
# normal equations
beta_normal = np.linalg.inv(X.T @ X) @ X.T @ y
print(beta_normal.round(rounding_level))

### Gradient Descent Algorithm

In [ ]:
def gradient_descent(X, y, eta=0.01, epochs=500, initial_beta=None):
    """
    Perform gradient descent for linear regression.
    
    Parameters:
    -----------
    X : array-like, shape (n_samples, n_features)
        Feature matrix with intercept column
    y : array-like, shape (n_samples, 1)
        Target values
    eta : float, default=0.01
        Learning rate
    epochs : int, default=500
        Number of iterations
    initial_beta : array-like, optional
        Initial coefficient values. If None, initialized to zeros.
    
    Returns:
    --------
    beta_gd : array, shape (n_features, 1)
        Final coefficient values
    save_betas : array, shape (epochs, n_features, 1)
        Coefficient values at each iteration
    cost : array, shape (epochs,)
        Cost function values at each iteration
    change : float
        Final change in coefficients
    """
    n = X.shape[0]
    p = X.shape[1]
    
    # Initialize coefficients
    if initial_beta is None:
        beta_gd = np.zeros((p, 1))
    else:
        beta_gd = initial_beta.copy()
    
    save_betas = []
    cost = []
    
    for e in range(epochs):
        save_betas.append(beta_gd.copy())
        gradients = (1/n) * X.T @ (X @ beta_gd - y)
        new_betas = beta_gd - eta * gradients
        change = np.abs(new_betas - beta_gd).sum()
        beta_gd = new_betas
        
        # Save the cost function value
        yhat = X @ beta_gd
        residuals = yhat - y
        cost.append((1/(2*n)) * residuals.T @ residuals)
    
    # Convert lists to numpy arrays
    save_betas = np.array(save_betas)
    cost = np.array(cost).flatten()
    
    return beta_gd, save_betas, cost, change


# Run gradient descent
beta_gd, save_betas, cost, change = gradient_descent(X, y, eta=0.01, epochs=500)

print(f"Final change: {change}")
print(f"Number of epochs: 500")
print(f"Final coefficients:\n{beta_gd}")

In [ ]:
# plot the cost function value at each iteration

b0_range = np.linspace(-5, 5, 100)
b1_range = np.linspace(-1, 1, 100)
B0, B1 = np.meshgrid(b0_range, b1_range)
Z = np.array([
    np.mean(((B0_val + B1_val * X[:, 1]) - y.ravel())**2)
    for B0_val, B1_val in zip(B0.ravel(), B1.ravel())
]).reshape(B0.shape)

plt.contourf(B0, B1, Z, levels=50, cmap="viridis")
plt.colorbar(label="Loss")
plt.title("Gradient Descent Trajectory")
plt.xlabel("Beta 0")
plt.ylabel("Beta 1")

plt.plot(save_betas[:, 0], save_betas[:, 1], 'r-o', label="Gradient Descent Path")
plt.legend()
plt.show()